# CS221, Spring 2019, PS2 Reconstruct
Haiyuan Mei (hmei0411@stanford.edu)

## Problem 1: word segmentation

### a. Show that the greedy search is suboptimal
In particular, provide an example input string on which the greedy approach would fail to find the lowest-cost segmentation of the input. 
* Example: upcomingeveningshow -> up coming even in g show   
  Should be 'upcoming evening show'. 

### b. Coding of the state-space search problem
